In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import cv2
import os

In [14]:

def load_images_from_folder(folder, csv_file):
    labels_df = pd.read_csv(csv_file)

    images = []
    labels = []
    
    valid_image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']
    
    for index, row in labels_df.iterrows():
        image_name = row['imageID']
        label = row['binaryLabels']
        
        _, file_extension = os.path.splitext(image_name)
        
        if file_extension.lower() in valid_image_extensions:

            img_path = os.path.join(folder, image_name)
            
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            img = cv2.resize(img, (512, 512))
            
            img = img / 255.0
            
            images.append(img)
            labels.append(label)
    
    images = np.array(images)
    labels = np.array(labels)
    
    images = images.reshape(-1, 512, 512, 1)  
    
    return images, labels

image_folder = "/Users/abhaykadambi/Desktop/School work/HackCMUProject/Images_Square"
csv_file = "/Users/abhaykadambi/Desktop/School work/HackCMUProject/G1020.csv"
x_data, y_data = load_images_from_folder(image_folder, csv_file)

split_index = int(0.8 * len(x_data))
x_train, x_test = x_data[:split_index], x_data[split_index:]
y_train, y_test = y_data[:split_index], y_data[split_index:]

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')       # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')


/Users/abhaykadambi/Desktop/School work/HackCMUProject/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6363 - loss: 1.0901
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.7119 - loss: 0.6153
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7262 - loss: 0.6121
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7276 - loss: 0.6211
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7107 - loss: 0.6188
7/7 - 2s - 298ms/step - accuracy: 0.7157 - loss: 0.6003

Test accuracy: 0.7156862616539001


In [11]:
model.save('./cancerDetector.keras')